In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr, pearsonr, ttest_ind

In [34]:
df = pd.read_csv('./data/Student_performance_data.csv')
df

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,3388,18,1,0,3,10.680555,2,0,4,1,0,0,0,3.455509,0.0
2388,3389,17,0,0,1,7.583217,4,1,4,0,1,0,0,3.279150,4.0
2389,3390,16,1,0,2,6.805500,20,0,2,0,0,0,1,1.142333,2.0
2390,3391,16,1,1,0,12.416653,17,0,2,0,1,1,0,1.803297,1.0


In [35]:
# Variables a correlacionar con 'GPA'
variables = ['Age', 'StudyTimeWeekly', 'Absences', 'Gender', 'Ethnicity', 'ParentalEducation', 'Tutoring', 'ParentalSupport', 'Extracurricular', 'Sports', 'Music', 'Volunteering']

In [36]:
# Calcular la correlación de Spearman
spearman_results = {}
for var in variables:
  correlation, p_value = spearmanr(df[var], df['GPA'])
  spearman_results[var] = {'Spearman Correlation': correlation, 'p_value': p_value}

# Mostrar resultados
spearman_results_df = pd.DataFrame(spearman_results).transpose().sort_values(by='Spearman Correlation')
spearman_results_df

,Spearman Correlation,p_value
Absences,-0.925467,0.000000e+00
ParentalEducation,-0.034648,9.022922e-02
Gender,-0.014361,4.826459e-01
Age,-0.002075,9.192252e-01
Volunteering,0.000897,9.650430e-01
Ethnicity,0.031724,1.208722e-01
Sports,0.051180,1.229921e-02
Music,0.068787,7.613688e-04
Extracurricular,0.086506,2.269565e-05
Tutoring,0.135122,3.233357e-11


In [37]:
# Calcular la correlación de Pearson
pearson_results = {}
for var in variables:
  correlation, p_value = pearsonr(df[var], df['GPA'])
  pearson_results[var] = {'Pearson Correlation': correlation, 'p_value': p_value}

# Mostrar resultados
pearson_results_df = pd.DataFrame(pearson_results).transpose().sort_values(by='Pearson Correlation')
pearson_results_df

,Pearson Correlation,p_value
Absences,-0.919314,0.000000e+00
ParentalEducation,-0.035854,7.957118e-02
Gender,-0.013360,5.136929e-01
Age,0.000275,9.892595e-01
Volunteering,0.003258,8.734676e-01
Ethnicity,0.027760,1.747114e-01
Sports,0.057859,4.645076e-03
Music,0.073318,3.322056e-04
Extracurricular,0.094078,4.045914e-06
Tutoring,0.145119,9.923854e-13


In [38]:
# Calcular la correlación de Pearson
ttest_ind_results = {}
for var in variables:
  correlation, p_value = ttest_ind(df[var], df['GPA'])
  ttest_ind_results[var] = {'ttest_ind Correlation': correlation, 'p_value': p_value}

# Mostrar resultados
ttest_ind_results_df = pd.DataFrame(ttest_ind_results).transpose().sort_values(by='ttest_ind Correlation')
ttest_ind_results_df

,ttest_ind Correlation,p_value
Volunteering,-86.850607,0.000000e+00
Music,-83.777468,0.000000e+00
Tutoring,-76.661484,0.000000e+00
Sports,-76.531473,0.000000e+00
Extracurricular,-71.866679,0.000000e+00
Gender,-65.439421,0.000000e+00
Ethnicity,-36.544681,4.345415e-258
ParentalEducation,-5.769653,8.443931e-09
ParentalSupport,7.289240,3.628739e-13
StudyTimeWeekly,67.180625,0.000000e+00
